In [3]:
%load_ext autoreload
%autoreload 2

%matplotlib inline

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [4]:
import numpy as np
import pandas as pd
from pathlib import Path
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import KFold
import lightgbm as lgb

In [5]:
path=Path('/kaggle/data_science_bowl')
path

PosixPath('/kaggle/data_science_bowl')

### Read Data

In [6]:
def read_data():
    train_df = pd.read_csv(path/'train.csv')
    test_df = pd.read_csv(path/'test.csv')
    train_labels_df = pd.read_csv(path/'train_labels.csv')
    specs_df = pd.read_csv(path/'specs.csv')
    return train_df, test_df, train_labels_df, specs_df

In [7]:
train_df, test_df, train_labels_df, specs_df = read_data()

In [8]:
train_df.head()

,event_id,game_session,timestamp,event_data,installation_id,event_count,event_code,game_time,title,type,world
0,27253bdc,45bb1e1b6b50c07b,2019-09-06T17:53:46.937Z,"{""event_code"": 2000, ""event_count"": 1}",0001e90f,1,2000,0,Welcome to Lost Lagoon!,Clip,NONE
1,27253bdc,17eeb7f223665f53,2019-09-06T17:54:17.519Z,"{""event_code"": 2000, ""event_count"": 1}",0001e90f,1,2000,0,Magma Peak - Level 1,Clip,MAGMAPEAK
2,77261ab5,0848ef14a8dc6892,2019-09-06T17:54:56.302Z,"{""version"":""1.0"",""event_count"":1,""game_time"":0...",0001e90f,1,2000,0,Sandcastle Builder (Activity),Activity,MAGMAPEAK
3,b2dba42b,0848ef14a8dc6892,2019-09-06T17:54:56.387Z,"{""description"":""Let's build a sandcastle! Firs...",0001e90f,2,3010,53,Sandcastle Builder (Activity),Activity,MAGMAPEAK
4,1bb5fbdb,0848ef14a8dc6892,2019-09-06T17:55:03.253Z,"{""description"":""Let's build a sandcastle! Firs...",0001e90f,3,3110,6972,Sandcastle Builder (Activity),Activity,MAGMAPEAK


In [9]:
test_df.head()

,event_id,game_session,timestamp,event_data,installation_id,event_count,event_code,game_time,title,type,world
0,27253bdc,0ea9ecc81a565215,2019-09-10T16:50:24.910Z,"{""event_code"": 2000, ""event_count"": 1}",00abaee7,1,2000,0,Welcome to Lost Lagoon!,Clip,NONE
1,27253bdc,c1ea43d8b8261d27,2019-09-10T16:50:55.503Z,"{""event_code"": 2000, ""event_count"": 1}",00abaee7,1,2000,0,Magma Peak - Level 1,Clip,MAGMAPEAK
2,27253bdc,7ed86c6b72e725e2,2019-09-10T16:51:51.805Z,"{""event_code"": 2000, ""event_count"": 1}",00abaee7,1,2000,0,Magma Peak - Level 2,Clip,MAGMAPEAK
3,27253bdc,7e516ace50e7fe67,2019-09-10T16:53:12.825Z,"{""event_code"": 2000, ""event_count"": 1}",00abaee7,1,2000,0,Crystal Caves - Level 1,Clip,CRYSTALCAVES
4,7d093bf9,a022c3f60ba547e7,2019-09-10T16:54:12.115Z,"{""version"":""1.0"",""round"":0,""event_count"":1,""ga...",00abaee7,1,2000,0,Chow Time,Game,CRYSTALCAVES


In [10]:
train_labels_df.head(10)

,game_session,installation_id,title,num_correct,num_incorrect,accuracy,accuracy_group
0,6bdf9623adc94d89,0006a69f,Mushroom Sorter (Assessment),1,0,1.0,3
1,77b8ee947eb84b4e,0006a69f,Bird Measurer (Assessment),0,11,0.0,0
2,901acc108f55a5a1,0006a69f,Mushroom Sorter (Assessment),1,0,1.0,3
3,9501794defd84e4d,0006a69f,Mushroom Sorter (Assessment),1,1,0.5,2
4,a9ef3ecb3d1acc6a,0006a69f,Bird Measurer (Assessment),1,0,1.0,3
5,197a373a77101924,0006c192,Cauldron Filler (Assessment),1,0,1.0,3
6,957406a905d59afd,0006c192,Bird Measurer (Assessment),1,1,0.5,2
7,b2297d292892745a,0006c192,Mushroom Sorter (Assessment),0,4,0.0,0
8,ae691ec5ad5652cf,00129856,Bird Measurer (Assessment),1,0,1.0,3
9,7b536271e99518f0,001d0ed0,Bird Measurer (Assessment),0,5,0.0,0


In [11]:
train_labels_df.shape

(17690, 7)

In [12]:
train_labels_df.groupby(['game_session', 'installation_id']).count()

,,title,num_correct,num_incorrect,accuracy,accuracy_group
game_session,installation_id,,,,,
00097cda27afb726,01bdd720,1,1,1,1,1
000f68cff32664ef,3f0dca37,1,1,1,1,1
0014403daadf67aa,29d1aaee,1,1,1,1,1
0014daa1d3e26eb2,55fdf49f,1,1,1,1,1
001c49e9e9968dbe,19d4b097,1,1,1,1,1
...,...,...,...,...,...,...
ffe9dd5aa0bf9231,53858b6a,1,1,1,1,1
fff20fa828fb1a94,5fb6ef75,1,1,1,1,1
fff441b089d7b1ef,54d91d26,1,1,1,1,1


In [13]:
train_df[train_df['game_session'] == '6bdf9623adc94d89'].merge(train_labels_df, on=['game_session'])

,event_id,game_session,timestamp,event_data,installation_id_x,event_count,event_code,game_time,title_x,type,world,installation_id_y,title_y,num_correct,num_incorrect,accuracy,accuracy_group
0,3bfd1a65,6bdf9623adc94d89,2019-08-06T05:37:50.020Z,"{""version"":""1.0"",""event_count"":1,""game_time"":0...",0006a69f,1,2000,0,Mushroom Sorter (Assessment),Assessment,TREETOPCITY,0006a69f,Mushroom Sorter (Assessment),1,0,1.0,3
1,db02c830,6bdf9623adc94d89,2019-08-06T05:37:50.078Z,"{""event_count"":2,""game_time"":77,""event_code"":2...",0006a69f,2,2025,77,Mushroom Sorter (Assessment),Assessment,TREETOPCITY,0006a69f,Mushroom Sorter (Assessment),1,0,1.0,3
2,a1e4395d,6bdf9623adc94d89,2019-08-06T05:37:50.082Z,"{""description"":""Pull three mushrooms out of th...",0006a69f,3,3010,77,Mushroom Sorter (Assessment),Assessment,TREETOPCITY,0006a69f,Mushroom Sorter (Assessment),1,0,1.0,3
3,7da34a02,6bdf9623adc94d89,2019-08-06T05:37:52.799Z,"{""coordinates"":{""x"":199,""y"":484,""stage_width"":...",0006a69f,4,4070,2784,Mushroom Sorter (Assessment),Assessment,TREETOPCITY,0006a69f,Mushroom Sorter (Assessment),1,0,1.0,3
4,28ed704e,6bdf9623adc94d89,2019-08-06T05:37:53.631Z,"{""height"":1,""coordinates"":{""x"":171,""y"":519,""st...",0006a69f,5,4025,3625,Mushroom Sorter (Assessment),Assessment,TREETOPCITY,0006a69f,Mushroom Sorter (Assessment),1,0,1.0,3
5,a52b92d5,6bdf9623adc94d89,2019-08-06T05:37:53.632Z,"{""description"":""Pull three mushrooms out of th...",0006a69f,6,3110,3625,Mushroom Sorter (Assessment),Assessment,TREETOPCITY,0006a69f,Mushroom Sorter (Assessment),1,0,1.0,3
6,9d29771f,6bdf9623adc94d89,2019-08-06T05:37:53.635Z,"{""description"":""That's one!"",""identifier"":""Dot...",0006a69f,7,3021,3625,Mushroom Sorter (Assessment),Assessment,TREETOPCITY,0006a69f,Mushroom Sorter (Assessment),1,0,1.0,3
7,c74f40cd,6bdf9623adc94d89,2019-08-06T05:37:54.253Z,"{""description"":""That's one!"",""identifier"":""Dot...",0006a69f,8,3121,4250,Mushroom Sorter (Assessment),Assessment,TREETOPCITY,0006a69f,Mushroom Sorter (Assessment),1,0,1.0,3
8,28ed704e,6bdf9623adc94d89,2019-08-06T05:37:54.930Z,"{""height"":2,""coordinates"":{""x"":496,""y"":502,""st...",0006a69f,9,4025,4925,Mushroom Sorter (Assessment),Assessment,TREETOPCITY,0006a69f,Mushroom Sorter (Assessment),1,0,1.0,3
9,9d29771f,6bdf9623adc94d89,2019-08-06T05:37:54.934Z,"{""description"":""two..."",""identifier"":""Dot_Two""...",0006a69f,10,3021,4925,Mushroom Sorter (Assessment),Assessment,TREETOPCITY,0006a69f,Mushroom Sorter (Assessment),1,0,1.0,3


In [96]:
import json

def process_event_code(x, is_correct):
    count = 0
    for xi in x:
        fields = json.loads(xi)
        if 'event_code' in fields and fields['event_code'] in [4100, 4110] and 'correct' in fields and fields['correct'] == is_correct:
            count += 1
    return count

def process_correct_event_code(x):
    return process_event_code(x, True)

def process_incorrect_event_code(x):
    return process_event_code(x, False)

def extract_correct_incorrect(df, field_name, func):
    key = ['installation_id', 'game_session']
    event_code_count = df[:].groupby(key)['event_data'].agg(func)
    event_code_count = event_code_count.reset_index()
    event_code_count.columns = [*key, field_name]
    event_code_count[field_name].fillna(0.0, inplace=True)
    return event_code_count

def extract_correct(df):
    return extract_correct_incorrect(df, 'num_correct', process_correct_event_code)

def extract_incorrect(df):
    return extract_correct_incorrect(df, 'num_incorrect', process_incorrect_event_code)

In [97]:
event_code_correct = extract_correct(train_df)

In [98]:
event_code_correct[event_code_correct['num_correct'] > 1]

,installation_id,game_session,num_correct
55,0006a69f,8043825259dc7ddd,2
68,0006a69f,a9ef3ecb3d1acc6a,2
172,00129856,ae691ec5ad5652cf,2
407,002db7e3,3a7e81ebad2a1c2d,5
413,002db7e3,5025f22d6e944533,2
...,...,...,...
302897,ff9715db,61ff809d90f3c4de,4
303046,ffc90c32,53f731c7dba9cc6d,3
303053,ffc90c32,7bde486376369bab,8
303067,ffc90c32,c996482b11d149dd,2


In [99]:
event_code_incorrect = extract_incorrect(train_df)

In [100]:
event_code_incorrect

,installation_id,game_session,num_incorrect
0,0001e90f,04168b61c0650977,0
1,0001e90f,07bacda7f9437b38,0
2,0001e90f,0848ef14a8dc6892,0
3,0001e90f,091587ed87b9637c,0
4,0001e90f,17eeb7f223665f53,0
...,...,...,...
303314,fffc0583,c09b94eebfdf50a6,0
303315,fffc0583,da9cabb5cb123df1,0
303316,fffc0583,e131c218f77fbeab,0
303317,fffc0583,e6b760e8f9e40ebf,0


In [15]:
event_code_count.reset_index(inplace=True)

In [222]:
correct_events_df = event_code_count[(event_code_count['event_code'] == 4100) | (event_code_count['event_code'] == 4110)]

In [223]:
del correct_events_df['event_code']

In [214]:
correct_events_df.columns = ['installation_id', 'game_session', 'num_correct']

In [215]:
correct_events_df

,installation_id,game_session,num_correct
291,0006a69f,6bdf9623adc94d89,1
323,0006a69f,77b8ee947eb84b4e,11
349,0006a69f,8043825259dc7ddd,4
364,0006a69f,80d34a30c2998653,4
365,0006a69f,80d34a30c2998653,2
...,...,...,...
1446339,ffeb0b1b,5448d652309a6324,3
1446367,ffeb0b1b,a6885ab824fbc32c,1
1446424,ffeb0b1b,dadd1a4d8ac68ab0,3
1446479,ffecb712,26b2ccaacf21b135,11


In [226]:
correct_events_df = extract_correct(test_df)
correct_events_df

,installation_id,game_session,num_correct
76,00abaee7,8b38fc0d2fd315dc,1
128,01242218,009c890ce6c4f3e3,2
169,01242218,16a00888451f6ff7,9
189,01242218,31423dbcd717919e,2
241,01242218,597a8839a5a3468d,2
...,...,...,...
147502,ffc73fb2,905d31d016c7a685,1
147518,ffc73fb2,b46072c5e2b3d8c5,1
147578,ffe00ca8,70336ec581799feb,2
147609,ffe00ca8,c116d9e6f8cf85c3,4


In [205]:
correct_events_df.pivot_table(index = 'installation_id', columns = ['event_code'], values = 'count').fillna(0)

event_code,4100,4110
installation_id,,
0006a69f,13.0,14.0
0006c192,6.0,2.0
00129856,1.0,1.0
001d0ed0,9.0,5.0
00225f67,0.0,2.0
...,...,...
ff9715db,28.0,0.0
ffc90c32,19.0,1.0
ffd2871d,2.0,0.0


In [270]:
def extract_correct_by_installation_id(df):
    event_code_count = df.groupby(['installation_id', 'event_code'])['event_code'].agg(['count'])
    event_code_count.reset_index(inplace=True)
    correct_events_df = event_code_count[(event_code_count['event_code'] == 4100) | (event_code_count['event_code'] == 4110)]
    del correct_events_df['event_code']
    correct_events_df.columns = ['installation_id', 'num_correct']
    correct_events_df = correct_events_df.groupby(['installation_id'])['num_correct'].agg(['sum'])
    correct_events_df.columns = ['num_correct']
    return correct_events_df

In [271]:
correct_by_installation_id = extract_correct_by_installation_id(train_df)

In [272]:
correct_by_installation_id

,num_correct
installation_id,
0006a69f,27
0006c192,8
00129856,2
001d0ed0,14
00225f67,2
...,...
ff9715db,28
ffc90c32,20
ffd2871d,2


In [263]:
correct_by_installation_id.columns = ['num_correct']
correct_by_installation_id

ValueError: Length mismatch: Expected axis has 2 elements, new values have 1 elements

In [19]:
main_key = 'installation_id'

In [20]:
last_group_df = test_df.groupby(main_key).last()

In [21]:
last_group_df

,event_id,game_session,timestamp,event_data,event_count,event_code,game_time,title,type,world
installation_id,,,,,,,,,,
00abaee7,90d848e0,348d7f09f96af313,2019-09-12T13:52:12.193Z,"{""version"":""1.0"",""event_count"":1,""game_time"":0...",1,2000,0,Cauldron Filler (Assessment),Assessment,MAGMAPEAK
01242218,7ad3efc6,1fef5d54cb4b775a,2019-10-09T20:23:16.209Z,"{""version"":""1.0"",""event_count"":1,""game_time"":0...",1,2000,0,Cart Balancer (Assessment),Assessment,CRYSTALCAVES
017c5718,3bfd1a65,4b165a330a0bdd6c,2019-09-21T11:28:21.757Z,"{""version"":""1.0"",""event_count"":1,""game_time"":0...",1,2000,0,Mushroom Sorter (Assessment),Assessment,TREETOPCITY
01a44906,3bfd1a65,be0b655ad1fee30c,2019-07-27T16:28:10.394Z,"{""version"":""1.0"",""event_count"":1,""game_time"":0...",1,2000,0,Mushroom Sorter (Assessment),Assessment,TREETOPCITY
01bc6cb6,7ad3efc6,46e8bbed71df7520,2019-09-06T18:05:26.197Z,"{""version"":""1.0"",""event_count"":1,""game_time"":0...",1,2000,0,Cart Balancer (Assessment),Assessment,CRYSTALCAVES
...,...,...,...,...,...,...,...,...,...,...
fee254cf,90d848e0,d0ea3550fd72f6c1,2019-09-28T01:44:50.390Z,"{""version"":""1.0"",""event_count"":1,""game_time"":0...",1,2000,0,Cauldron Filler (Assessment),Assessment,MAGMAPEAK
ff57e602,5b49460a,d22d3b1442967ba7,2019-10-13T17:18:24.776Z,"{""version"":""1.0"",""event_count"":1,""game_time"":0...",1,2000,0,Chest Sorter (Assessment),Assessment,CRYSTALCAVES
ffc73fb2,7ad3efc6,fd5e0fd3fe28f907,2019-10-07T20:36:58.937Z,"{""version"":""1.0"",""event_count"":1,""game_time"":0...",1,2000,0,Cart Balancer (Assessment),Assessment,CRYSTALCAVES


In [40]:
last_group_df[last_group_df['game_session'] == '348d7f09f96af313']['event_data']

installation_id
00abaee7    {"version":"1.0","event_count":1,"game_time":0...
Name: event_data, dtype: object

In [22]:
last_group_df['type'].unique()

array(['Assessment'], dtype=object)

In [23]:
last_group_df['title'].unique()

array(['Cauldron Filler (Assessment)', 'Cart Balancer (Assessment)',
       'Mushroom Sorter (Assessment)', 'Bird Measurer (Assessment)',
       'Chest Sorter (Assessment)'], dtype=object)

In [45]:
test_df_titles = list(test_df['title'].unique())
test_df_titles.sort()
test_df_titles

['12 Monkeys',
 'Air Show',
 'All Star Sorting',
 'Balancing Act',
 'Bird Measurer (Assessment)',
 'Bottle Filler (Activity)',
 'Bubble Bath',
 'Bug Measurer (Activity)',
 'Cart Balancer (Assessment)',
 'Cauldron Filler (Assessment)',
 'Chest Sorter (Assessment)',
 'Chicken Balancer (Activity)',
 'Chow Time',
 'Costume Box',
 'Crystal Caves - Level 1',
 'Crystal Caves - Level 2',
 'Crystal Caves - Level 3',
 'Crystals Rule',
 'Dino Dive',
 'Dino Drink',
 'Egg Dropper (Activity)',
 'Fireworks (Activity)',
 'Flower Waterer (Activity)',
 'Happy Camel',
 'Heavy, Heavier, Heaviest',
 'Honey Cake',
 'Leaf Leader',
 'Lifting Heavy Things',
 'Magma Peak - Level 1',
 'Magma Peak - Level 2',
 'Mushroom Sorter (Assessment)',
 'Ordering Spheres',
 'Pan Balance',
 "Pirate's Tale",
 'Rulers',
 'Sandcastle Builder (Activity)',
 'Scrub-A-Dub',
 'Slop Problem',
 'Treasure Map',
 'Tree Top City - Level 1',
 'Tree Top City - Level 2',
 'Tree Top City - Level 3',
 'Watering Hole (Activity)',
 'Welcome to 

In [24]:
comp_train_df = pd.DataFrame({main_key: train_df[main_key].unique()})
comp_train_df.set_index(main_key, inplace = True)

In [25]:
comp_train_df

""
installation_id
0001e90f
000447c4
0006a69f
0006c192
0009a5a9
...
fff64664
fff70c1e
fff744ce


In [46]:
list(train_df['world'].unique())

['NONE', 'MAGMAPEAK', 'TREETOPCITY', 'CRYSTALCAVES']

In [49]:
train_labels_df['title'].unique()

array(['Mushroom Sorter (Assessment)', 'Bird Measurer (Assessment)',
       'Cauldron Filler (Assessment)', 'Chest Sorter (Assessment)',
       'Cart Balancer (Assessment)'], dtype=object)

In [105]:
merged_train_df = train_df.merge(train_labels_df, on=['installation_id', 'game_session'], how='left')

In [107]:
merged_train_df.head(30)

,event_id,game_session,timestamp,event_data,installation_id,event_count,event_code,game_time,title_x,type,world,title_y,num_correct,num_incorrect,accuracy,accuracy_group
0,27253bdc,45bb1e1b6b50c07b,2019-09-06T17:53:46.937Z,"{""event_code"": 2000, ""event_count"": 1}",0001e90f,1,2000,0,Welcome to Lost Lagoon!,Clip,NONE,NaN,NaN,NaN,NaN,NaN
1,27253bdc,17eeb7f223665f53,2019-09-06T17:54:17.519Z,"{""event_code"": 2000, ""event_count"": 1}",0001e90f,1,2000,0,Magma Peak - Level 1,Clip,MAGMAPEAK,NaN,NaN,NaN,NaN,NaN
2,77261ab5,0848ef14a8dc6892,2019-09-06T17:54:56.302Z,"{""version"":""1.0"",""event_count"":1,""game_time"":0...",0001e90f,1,2000,0,Sandcastle Builder (Activity),Activity,MAGMAPEAK,NaN,NaN,NaN,NaN,NaN
3,b2dba42b,0848ef14a8dc6892,2019-09-06T17:54:56.387Z,"{""description"":""Let's build a sandcastle! Firs...",0001e90f,2,3010,53,Sandcastle Builder (Activity),Activity,MAGMAPEAK,NaN,NaN,NaN,NaN,NaN
4,1bb5fbdb,0848ef14a8dc6892,2019-09-06T17:55:03.253Z,"{""description"":""Let's build a sandcastle! Firs...",0001e90f,3,3110,6972,Sandcastle Builder (Activity),Activity,MAGMAPEAK,NaN,NaN,NaN,NaN,NaN
5,1325467d,0848ef14a8dc6892,2019-09-06T17:55:06.279Z,"{""coordinates"":{""x"":583,""y"":605,""stage_width"":...",0001e90f,4,4070,9991,Sandcastle Builder (Activity),Activity,MAGMAPEAK,NaN,NaN,NaN,NaN,NaN
6,1325467d,0848ef14a8dc6892,2019-09-06T17:55:06.913Z,"{""coordinates"":{""x"":601,""y"":570,""stage_width"":...",0001e90f,5,4070,10622,Sandcastle Builder (Activity),Activity,MAGMAPEAK,NaN,NaN,NaN,NaN,NaN
7,1325467d,0848ef14a8dc6892,2019-09-06T17:55:07.546Z,"{""coordinates"":{""x"":250,""y"":665,""stage_width"":...",0001e90f,6,4070,11255,Sandcastle Builder (Activity),Activity,MAGMAPEAK,NaN,NaN,NaN,NaN,NaN
8,1325467d,0848ef14a8dc6892,2019-09-06T17:55:07.979Z,"{""coordinates"":{""x"":279,""y"":629,""stage_width"":...",0001e90f,7,4070,11689,Sandcastle Builder (Activity),Activity,MAGMAPEAK,NaN,NaN,NaN,NaN,NaN
9,1325467d,0848ef14a8dc6892,2019-09-06T17:55:08.566Z,"{""coordinates"":{""x"":839,""y"":654,""stage_width"":...",0001e90f,8,4070,12272,Sandcastle Builder (Activity),Activity,MAGMAPEAK,NaN,NaN,NaN,NaN,NaN


In [73]:
merged_train_df[merged_train_df['title_y'].isnull()]

,event_id,game_session,timestamp,event_data,installation_id,event_count,event_code,game_time,title_x,type,world,title_y,num_correct,num_incorrect,accuracy,accuracy_group
0,27253bdc,45bb1e1b6b50c07b,2019-09-06T17:53:46.937Z,"{""event_code"": 2000, ""event_count"": 1}",0001e90f,1,2000,0,Welcome to Lost Lagoon!,Clip,NONE,NaN,NaN,NaN,NaN,NaN
1,27253bdc,17eeb7f223665f53,2019-09-06T17:54:17.519Z,"{""event_code"": 2000, ""event_count"": 1}",0001e90f,1,2000,0,Magma Peak - Level 1,Clip,MAGMAPEAK,NaN,NaN,NaN,NaN,NaN
2,77261ab5,0848ef14a8dc6892,2019-09-06T17:54:56.302Z,"{""version"":""1.0"",""event_count"":1,""game_time"":0...",0001e90f,1,2000,0,Sandcastle Builder (Activity),Activity,MAGMAPEAK,NaN,NaN,NaN,NaN,NaN
3,b2dba42b,0848ef14a8dc6892,2019-09-06T17:54:56.387Z,"{""description"":""Let's build a sandcastle! Firs...",0001e90f,2,3010,53,Sandcastle Builder (Activity),Activity,MAGMAPEAK,NaN,NaN,NaN,NaN,NaN
4,1bb5fbdb,0848ef14a8dc6892,2019-09-06T17:55:03.253Z,"{""description"":""Let's build a sandcastle! Firs...",0001e90f,3,3110,6972,Sandcastle Builder (Activity),Activity,MAGMAPEAK,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11341037,ab3136ba,c09b94eebfdf50a6,2019-10-10T15:11:15.584Z,"{""description"":""Tap another dinosaur."",""identi...",fffc0583,66,3110,118615,Dino Dive,Game,MAGMAPEAK,NaN,NaN,NaN,NaN,NaN
11341038,27253bdc,6ad8ab25003ef1b0,2019-10-10T15:12:17.364Z,"{""event_code"": 2000, ""event_count"": 1}",fffc0583,1,2000,0,Crystal Caves - Level 2,Clip,CRYSTALCAVES,NaN,NaN,NaN,NaN,NaN
11341039,27253bdc,71904dca23b982c6,2019-10-10T15:13:04.713Z,"{""event_code"": 2000, ""event_count"": 1}",fffc0583,1,2000,0,Crystal Caves - Level 3,Clip,CRYSTALCAVES,NaN,NaN,NaN,NaN,NaN
11341040,27253bdc,2267b18f069e875a,2019-10-10T15:13:54.545Z,"{""event_code"": 2000, ""event_count"": 1}",fffc0583,1,2000,0,Crystal Caves - Level 3,Clip,CRYSTALCAVES,NaN,NaN,NaN,NaN,NaN


In [108]:
sample_session_df = merged_train_df[merged_train_df['game_session'] == '0848ef14a8dc6892']

In [110]:
sample_session_df.head(40)

,event_id,game_session,timestamp,event_data,installation_id,event_count,event_code,game_time,title_x,type,world,title_y,num_correct,num_incorrect,accuracy,accuracy_group
2,77261ab5,0848ef14a8dc6892,2019-09-06T17:54:56.302Z,"{""version"":""1.0"",""event_count"":1,""game_time"":0...",0001e90f,1,2000,0,Sandcastle Builder (Activity),Activity,MAGMAPEAK,NaN,NaN,NaN,NaN,NaN
3,b2dba42b,0848ef14a8dc6892,2019-09-06T17:54:56.387Z,"{""description"":""Let's build a sandcastle! Firs...",0001e90f,2,3010,53,Sandcastle Builder (Activity),Activity,MAGMAPEAK,NaN,NaN,NaN,NaN,NaN
4,1bb5fbdb,0848ef14a8dc6892,2019-09-06T17:55:03.253Z,"{""description"":""Let's build a sandcastle! Firs...",0001e90f,3,3110,6972,Sandcastle Builder (Activity),Activity,MAGMAPEAK,NaN,NaN,NaN,NaN,NaN
5,1325467d,0848ef14a8dc6892,2019-09-06T17:55:06.279Z,"{""coordinates"":{""x"":583,""y"":605,""stage_width"":...",0001e90f,4,4070,9991,Sandcastle Builder (Activity),Activity,MAGMAPEAK,NaN,NaN,NaN,NaN,NaN
6,1325467d,0848ef14a8dc6892,2019-09-06T17:55:06.913Z,"{""coordinates"":{""x"":601,""y"":570,""stage_width"":...",0001e90f,5,4070,10622,Sandcastle Builder (Activity),Activity,MAGMAPEAK,NaN,NaN,NaN,NaN,NaN
7,1325467d,0848ef14a8dc6892,2019-09-06T17:55:07.546Z,"{""coordinates"":{""x"":250,""y"":665,""stage_width"":...",0001e90f,6,4070,11255,Sandcastle Builder (Activity),Activity,MAGMAPEAK,NaN,NaN,NaN,NaN,NaN
8,1325467d,0848ef14a8dc6892,2019-09-06T17:55:07.979Z,"{""coordinates"":{""x"":279,""y"":629,""stage_width"":...",0001e90f,7,4070,11689,Sandcastle Builder (Activity),Activity,MAGMAPEAK,NaN,NaN,NaN,NaN,NaN
9,1325467d,0848ef14a8dc6892,2019-09-06T17:55:08.566Z,"{""coordinates"":{""x"":839,""y"":654,""stage_width"":...",0001e90f,8,4070,12272,Sandcastle Builder (Activity),Activity,MAGMAPEAK,NaN,NaN,NaN,NaN,NaN
10,1325467d,0848ef14a8dc6892,2019-09-06T17:55:08.966Z,"{""coordinates"":{""x"":807,""y"":619,""stage_width"":...",0001e90f,9,4070,12672,Sandcastle Builder (Activity),Activity,MAGMAPEAK,NaN,NaN,NaN,NaN,NaN
11,1325467d,0848ef14a8dc6892,2019-09-06T17:55:09.673Z,"{""coordinates"":{""x"":810,""y"":611,""stage_width"":...",0001e90f,10,4070,13389,Sandcastle Builder (Activity),Activity,MAGMAPEAK,NaN,NaN,NaN,NaN,NaN


In [112]:
sample_session_df[sample_session_df['title_x'].str.contains("Assessment") == False]

,event_id,game_session,timestamp,event_data,installation_id,event_count,event_code,game_time,title_x,type,world,title_y,num_correct,num_incorrect,accuracy,accuracy_group
2,77261ab5,0848ef14a8dc6892,2019-09-06T17:54:56.302Z,"{""version"":""1.0"",""event_count"":1,""game_time"":0...",0001e90f,1,2000,0,Sandcastle Builder (Activity),Activity,MAGMAPEAK,NaN,NaN,NaN,NaN,NaN
3,b2dba42b,0848ef14a8dc6892,2019-09-06T17:54:56.387Z,"{""description"":""Let's build a sandcastle! Firs...",0001e90f,2,3010,53,Sandcastle Builder (Activity),Activity,MAGMAPEAK,NaN,NaN,NaN,NaN,NaN
4,1bb5fbdb,0848ef14a8dc6892,2019-09-06T17:55:03.253Z,"{""description"":""Let's build a sandcastle! Firs...",0001e90f,3,3110,6972,Sandcastle Builder (Activity),Activity,MAGMAPEAK,NaN,NaN,NaN,NaN,NaN
5,1325467d,0848ef14a8dc6892,2019-09-06T17:55:06.279Z,"{""coordinates"":{""x"":583,""y"":605,""stage_width"":...",0001e90f,4,4070,9991,Sandcastle Builder (Activity),Activity,MAGMAPEAK,NaN,NaN,NaN,NaN,NaN
6,1325467d,0848ef14a8dc6892,2019-09-06T17:55:06.913Z,"{""coordinates"":{""x"":601,""y"":570,""stage_width"":...",0001e90f,5,4070,10622,Sandcastle Builder (Activity),Activity,MAGMAPEAK,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
264,84538528,0848ef14a8dc6892,2019-09-06T17:58:05.792Z,"{""size"":3,""castles_placed"":[{""size"":2,""positio...",0001e90f,263,4020,189518,Sandcastle Builder (Activity),Activity,MAGMAPEAK,NaN,NaN,NaN,NaN,NaN
265,b2dba42b,0848ef14a8dc6892,2019-09-06T17:58:05.793Z,"{""description"":""So cool!"",""identifier"":""Dot_So...",0001e90f,264,3010,189518,Sandcastle Builder (Activity),Activity,MAGMAPEAK,NaN,NaN,NaN,NaN,NaN
266,1bb5fbdb,0848ef14a8dc6892,2019-09-06T17:58:07.009Z,"{""description"":""So cool!"",""identifier"":""Dot_So...",0001e90f,265,3110,190735,Sandcastle Builder (Activity),Activity,MAGMAPEAK,NaN,NaN,NaN,NaN,NaN
267,5e812b27,0848ef14a8dc6892,2019-09-06T17:58:07.423Z,"{""size"":0,""coordinates"":{""x"":782,""y"":207,""stag...",0001e90f,266,4030,191135,Sandcastle Builder (Activity),Activity,MAGMAPEAK,NaN,NaN,NaN,NaN,NaN


In [114]:
def unique_values(data):
    total = data.count()
    tt = pd.DataFrame(total)
    tt.columns = ['Total']
    uniques = []
    for col in data.columns:
        unique = data[col].nunique()
        uniques.append(unique)
    tt['Uniques'] = uniques
    return(np.transpose(tt))

In [116]:
unique_values(train_df)

,event_id,game_session,timestamp,event_data,installation_id,event_count,event_code,game_time,title,type,world
Total,11341042,11341042,11341042,11341042,11341042,11341042,11341042,11341042,11341042,11341042,11341042
Uniques,384,303319,10777308,10893749,17000,3368,42,834048,44,4,4


In [117]:
comp_train_df = pd.DataFrame({'installation_id': train_df['installation_id'].unique()})
comp_train_df.set_index('installation_id', inplace = True)

In [119]:
comp_train_df

""
installation_id
0001e90f
000447c4
0006a69f
0006c192
0009a5a9
...
fff64664
fff70c1e
fff744ce


In [149]:
agg_stats = ['mean', 'sum', 'min', 'max', 'std', 'skew', 'median', pd.Series.kurt, lambda x : np.corrcoef(x), 'count']

In [157]:
def get_numeric_columns(df, column):
    df = df.groupby('installation_id').agg({f'{column}': agg_stats})
    df[column].fillna(df[column].mean(), inplace = True)
    df.columns = [f'{column}_{stat}' for stat in agg_stats]
    df.rename(columns={ df.columns[7]: f'{column}_kurt' }, inplace = True)
    df.rename(columns={ df.columns[8]: f'{column}_corrcoef' }, inplace = True)
    df.fillna(0, inplace = True)
    return df

In [158]:
get_numeric_columns(train_df, 'game_time')

,game_time_mean,game_time_sum,game_time_min,game_time_max,game_time_std,game_time_skew,game_time_median,game_time_kurt,game_time_corrcoef,game_time_count
installation_id,,,,,,,,,,
0001e90f,201787.799558,273826044,0,570396,164377.067566,0.664036,150032.0,-0.836821,1.0,1357
000447c4,81011.751381,14663127,0,175347,47014.118535,0.101741,81525.0,-0.937713,1.0,181
0006a69f,82491.007366,313548319,0,1520600,159789.192183,7.504204,50155.0,62.680316,1.0,3801
0006c192,90398.352968,201045937,0,395901,77779.338412,1.366377,69533.0,1.873374,1.0,2224
0009a5a9,398593.543689,164220540,0,1684228,648848.534146,1.312675,60399.5,-0.262822,1.0,412
...,...,...,...,...,...,...,...,...,...,...
fff64664,516205.468549,402124060,0,1548496,582787.501688,0.650020,93563.0,-1.241505,1.0,779
fff70c1e,82465.818182,3628496,0,152279,57657.161595,-0.222848,91783.5,-1.648052,1.0,44
fff744ce,0.000000,0,0,0,0.000000,0.000000,0.0,0.000000,0.0,3
